In [2]:
import utils
import json
import plotly.express as px
import pandas as pd


with open('sub_codelists.json', 'r') as f:
    sub_codelists = json.load(f)

In [46]:
nrg_bal_constraints = {
    'freq': 'A',
    'nrg_bal': ['FC_E','FC_OTH_HH_E'],
    'siec': ['E7000','G3000'],
    'unit': 'GWH',
    'geo': sub_codelists['geo_eu_countries'],
}

#We are interested in: https://data.ecb.europa.eu/data/datasets/MNA/MNA.A.N.AT.W2.S1.S1.B.B1GQ._Z._Z._Z.EUR.V.N

mna_constraints = {
    "FREQ": "A",
    "ADJUSTMENT": "N",
    "REF_AREA":  sub_codelists['geo_eu_countries'],
    "COUNTERPART_AREA": "W2",
    "REF_SECTOR": "S1",
    "COUNTERPART_SECTOR": "S1",
    "ACCOUNTING_ENTRY": "B",
    "STO": "B1GQ",
    "INSTR_ASSET": "_Z",
    "ACTIVITY": "_Z",
    "EXPENDITURE": "_Z",
    "UNIT_MEASURE": "EUR",
    "PRICES": "V",
    "TRANSFORMATION": "N"
}


#nrg_prices do not have values for CY, FI, MT. Therefore, we need to drop them from the geo list
geo_list = [country for country in sub_codelists['geo_eu_countries'] if country not in ['CY', 'FI', 'MT']]


nrg_prices_gas_constraints = {
    'freq': 'S',
    "product": '',
    'nrg_cons': '',
    'unit':'KWH',
    'tax': ['I_TAX', 'X_TAX', 'X_VAT'],
    'currency':'EUR',
    'geo':geo_list
}

nrg_pices_electricity_constraints = {
    'freq': 'S',
    'product': '',
    'nrg_cons': '',
    'unit': 'KWH',
    'tax': ['I_TAX', 'X_TAX', 'X_VAT'],
    'currency':'EUR',
    'geo':geo_list
}

In [47]:
gdp = utils.get_dataset_with_selection('MNA', mna_constraints, agency='ecb')
nrg_bal = utils.get_dataset_with_selection('nrg_bal_s', nrg_bal_constraints, agency='estat')
gas_prices_hh = utils.get_dataset_with_selection('nrg_pc_202', nrg_prices_gas_constraints, agency='estat')
gas_prices_nhh = utils.get_dataset_with_selection('nrg_pc_203', nrg_prices_gas_constraints, agency='estat')
electricity_prices_hh = utils.get_dataset_with_selection('nrg_pc_204', nrg_pices_electricity_constraints, agency='estat')
electricity_prices_nhh = utils.get_dataset_with_selection('nrg_pc_205', nrg_pices_electricity_constraints, agency='estat')

data_url: https://data-api.ecb.europa.eu/service/data/MNA/A.N.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE.W2.S1.S1.B.B1GQ._Z._Z._Z.EUR.V.N?&startPeriod=2000
metadata_url: https://data-api.ecb.europa.eu/service/dataflow/all/MNA/latest?detail=full&references=descendants
data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nrg_bal_s/A.FC_E+FC_OTH_HH_E.E7000+G3000.GWH.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/nrg_bal_s/latest?detail=full&references=descendants
data_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nrg_pc_202/S...KWH.I_TAX+X_TAX+X_VAT.EUR.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+LV+LT+LU+HU+NL+AT+PL+PT+RO+SI+SK+SE?format=SDMX-CSV&startPeriod=2000
metadata_url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/all/nrg_pc_202/latest?detail=

In [79]:
from math import e
import pandas as pd

gdp_df = gdp.data[["REF_AREA", "TIME_PERIOD", "OBSVALUE", "UNIT_MULT"]]
gdp_df.loc[:, "OBSVALUE"] = gdp_df["OBSVALUE"].astype(float) * (10 ** gdp_df["UNIT_MULT"].astype(int))
gdp_df = gdp_df[["REF_AREA", "TIME_PERIOD", "OBSVALUE"]]

nrg_bal_df = nrg_bal.data

nrg_bal_df_hh = nrg_bal_df[nrg_bal_df["nrg_bal"] == "FC_OTH_HH_E"]
nrg_bal_df_hh = nrg_bal_df_hh.drop(columns=["nrg_bal"])
nrg_bal_df_hh.rename(columns={"OBS_VALUE": "HH_VALUE"}, inplace=True)

nrg_bal_df_total = nrg_bal_df[nrg_bal_df["nrg_bal"] == "FC_E"]
nrg_bal_df_total = nrg_bal_df_total.drop(columns=["nrg_bal"])
nrg_bal_df_total.rename(columns={"OBS_VALUE": "TOTAL_VALUE"}, inplace=True)

nrg_bal_df_nhh = pd.merge(nrg_bal_df_hh, nrg_bal_df_total, on=['siec', 'geo', 'TIME_PERIOD'])
nrg_bal_df_nhh["NHH_VALUE"] = nrg_bal_df_nhh["TOTAL_VALUE"] - nrg_bal_df_nhh["HH_VALUE"]
nrg_bal_df_nhh = nrg_bal_df_nhh[['siec', 'geo', 'TIME_PERIOD', "NHH_VALUE"]]

nrg_bal_df_hh.rename(columns={"HH_VALUE": "OBS_VALUE"}, inplace=True)
nrg_bal_df_hh["nrg_bal"] = "FC_OTH_HH_E"
nrg_bal_df_nhh.rename(columns={"NHH_VALUE": "OBS_VALUE"}, inplace=True)
nrg_bal_df_nhh["nrg_bal"] = "FC_NHH_E"

nrg_bal_df = pd.concat([nrg_bal_df_hh, nrg_bal_df_nhh], ignore_index=True)
nrg_bal_df.rename(columns={"OBS_VALUE": "GWH consumption"}, inplace=True)


gas_prices_hh_df = gas_prices_hh.data
gas_prices_hh_df['nrg_bal'] = 'FC_OTH_HH_E'
gas_prices_hh_df['siec'] = 'G3000'

gas_prices_nhh_df = gas_prices_nhh.data
gas_prices_nhh_df['nrg_bal'] = 'FC_NHH_E'
gas_prices_nhh_df['siec'] = 'G3000'

electricity_prices_hh_df = electricity_prices_hh.data
electricity_prices_hh_df['nrg_bal'] = 'FC_OTH_HH_E'
electricity_prices_hh_df['siec'] = 'E7000'

electricity_prices_nhh_df = electricity_prices_nhh.data
electricity_prices_nhh_df['nrg_bal'] = 'FC_NHH_E'
electricity_prices_nhh_df['siec'] = 'E7000'

energy_prices_df = pd.concat([gas_prices_hh_df, gas_prices_nhh_df, electricity_prices_hh_df, electricity_prices_nhh_df], ignore_index=True)

energy_prices_df = energy_prices_df.drop(columns=['OBS_FLAG', 'CONF_STATUS'])
energy_prices_df['TIME_PERIOD'] = energy_prices_df['TIME_PERIOD'].str[0:4].astype(int)
energy_prices_df = energy_prices_df.groupby(['geo', 'TIME_PERIOD', 'nrg_bal', 'siec', 'tax'])['OBS_VALUE'].mean().reset_index()
energy_prices_df = pd.pivot_table(energy_prices_df, values='OBS_VALUE', index=['geo', 'TIME_PERIOD', 'nrg_bal', 'siec'], columns=['tax'])
energy_prices_df.reset_index(inplace=True)
energy_prices_df['% tax'] = round((energy_prices_df['I_TAX'] - energy_prices_df['X_TAX']) / energy_prices_df['I_TAX'] * 100,2)
energy_prices_df.rename(columns={"I_TAX": "EUR per Kwh Including tax", "X_TAX": "EUR per Kwh Excluding tax", "X_VAT": "EUR per Kwh Excluding VAT and other recoverable taxes"}, inplace=True)
display(energy_prices_df)


tax,geo,TIME_PERIOD,nrg_bal,siec,EUR per Kwh Including tax,EUR per Kwh Excluding tax,EUR per Kwh Excluding VAT and other recoverable taxes,% tax
0,AT,2007,FC_NHH_E,E7000,0.114833,0.079550,0.095683,30.73
1,AT,2007,FC_NHH_E,G3000,0.053000,0.037950,0.044150,28.40
2,AT,2007,FC_OTH_HH_E,E7000,0.190100,0.135580,0.159800,28.68
3,AT,2007,FC_OTH_HH_E,G3000,0.063333,0.046467,0.052800,26.63
4,AT,2008,FC_NHH_E,E7000,0.123200,0.086908,0.102677,29.46
...,...,...,...,...,...,...,...,...
1709,SK,2023,FC_OTH_HH_E,G3000,0.090333,0.075283,0.075283,16.66
1710,SK,2024,FC_NHH_E,E7000,0.223362,0.163925,0.186125,26.61
1711,SK,2024,FC_NHH_E,G3000,0.088400,0.072360,0.073660,18.14
1712,SK,2024,FC_OTH_HH_E,E7000,0.189233,0.132200,0.157700,30.14


In [74]:
electricity_prices_hh_df = energy_prices_df[
    (energy_prices_df['nrg_bal'] == 'FC_NHH_E') &
    (energy_prices_df['siec'] == 'G3000') 
]

fig = px.line(electricity_prices_hh_df, x='TIME_PERIOD', y='% tax', color='geo')
fig.show()

In [71]:
electricity_prices_hh_df = energy_prices_df[
    (energy_prices_df['nrg_bal'] == 'FC_NHH_E') &
    (energy_prices_df['siec'] == 'G3000') 
]

fig = px.line(electricity_prices_hh_df, x='TIME_PERIOD', y='EUR per Kwh Excluding tax', color='geo')
fig.show()

In [93]:
prices_and_comsumption = pd.merge(energy_prices_df, nrg_bal_df, on=['geo', 'TIME_PERIOD', 'siec', 'nrg_bal'])
prices_and_comsumption['total_consumption'] = prices_and_comsumption['GWH consumption'] * prices_and_comsumption['EUR per Kwh Excluding tax'] * 10**6

prices_and_consumption_gas = prices_and_comsumption[prices_and_comsumption['siec'] == 'G3000']
cost_of_gas = prices_and_consumption_gas.groupby(['geo', 'TIME_PERIOD'])['total_consumption'].sum().reset_index()

gdp_df.rename(columns={"OBSVALUE": "GDP", "REF_AREA": "geo"}, inplace=True)

cost_of_gas_over_gdp = pd.merge(cost_of_gas, gdp_df, on=['geo', 'TIME_PERIOD'])
cost_of_gas_over_gdp['cost_of_gas_over_gdp'] = cost_of_gas_over_gdp['total_consumption'] / cost_of_gas_over_gdp['GDP']

fig = px.line(cost_of_gas_over_gdp, x='TIME_PERIOD', y='cost_of_gas_over_gdp', color='geo')
fig.show()




In [97]:
prices_and_consumption_electricity = prices_and_comsumption[prices_and_comsumption['siec'] == 'E7000']
cost_of_electricity = prices_and_consumption_electricity.groupby(['geo', 'TIME_PERIOD'])['total_consumption'].sum().reset_index()

cost_of_electricity_over_gdp = pd.merge(cost_of_electricity, gdp_df, on=['geo', 'TIME_PERIOD'])
cost_of_electricity_over_gdp['cost_of_electricity_over_gdp'] = cost_of_electricity_over_gdp['total_consumption'] / cost_of_electricity_over_gdp['GDP']

fig = px.line(cost_of_electricity_over_gdp, x='TIME_PERIOD', y='cost_of_electricity_over_gdp', color='geo')
fig.show()
